# Inverse Document Frequency

### Introduction

In this lesson, we'll begin our journey with NLP.  Our first step is to understand some of the terms that will allow identify the components of an NLP problem.  Let's get started.

### Loading our Data

Let's get started by loading up some of our newsgroups dataset from sklearn.

### Inverse Document Frequency

We can do a little bit better if in addition to common words, we also promote rare words.

In [175]:
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(document_tokens)

In [225]:
len(tokens)

In [278]:
def count_values(document_tokens):
    tokens = [token for doc in document_tokens for token in set(doc)]
    counter_full = dictionary.doc2bow(tokens, allow_update=True)
    idx_counts = dict(counter_full)
    values = idx_counts.values()
    return values

In [279]:
import numpy as np
def doc_frequency(document_tokens):
    count_values(document_tokens)
    freq_pct = np.array(list(values))/len(document_tokens)
    return np.around(freq_pct, decimals = 5)

In [289]:
def inverse_doc_frequency(document_tokens):
    values = count_values(document_tokens)
    length = len(document_tokens)
    inverse_freq = np.array(length/np.array(list(values)) + 1)
    return np.around(inverse_freq, decimals = 1)

In [290]:
idf = inverse_doc_frequency(document_tokens)

In [291]:
idf

array([   54.6,    36.1,  2829.5, ..., 11315. , 11315. , 11315. ])

Now what we would like to do is use the formula $idf*tf$, to reward words that are rare in the corpus but occur commonly in a particular document.  The problem right now, is that the idf score dominates the term frequency score.  Take a look at the numbers below.

In [ ]:
# {6: ['car', 4, 53.6, 214.4],
#  19: ['it', 2, 35.1, 70.2],
#  0: ['addition', 1, 2828.5, 2828.5],
#  1: ['body', 1, 49.4, 49.4],
#  2: ['bricklin', 1, 390.1, 390.1],
#  3: ['brought', 1, 13.6, 13.6],
#  4: ['bumper', 1, 19.9, 19.9],
#  5: ['called', 1, 11.9, 11.9],

We can fix by taking the log of the idf.  We'll update our formula accordingly.

In [5]:
# import numpy as np
# np.log10

In [6]:
def inverse_doc_frequency(document_tokens):
    values = count_values(document_tokens)
    length = len(document_tokens)
    inverse_freq = np.log10(np.array(length/np.array(list(values)) + 1))
    return np.around(inverse_freq, decimals = 1)

Now we'll let's see how this affects our word scores.

In [344]:
def calculate_tfidf(vector, document_tokens, top = 20):
    idf = inverse_doc_frequency(document_tokens)
    word_scores = [[idx, [dictionary.get(idx), num, inverse, num*inverse]] for (idx, num), inverse in zip(vector, idf)][:top]
    word_scores.sort(key=lambda x: x[-1][-1], reverse=True)
    return word_scores

In [354]:
top_words_first_doc[:10]

[[6, ['car', 4, 4.0, 16.0]],
 [0, ['addition', 1, 7.9, 7.9]],
 [12, ['enlighten', 1, 7.3, 7.3]],
 [19, ['it', 2, 3.6, 7.2]],
 [2, ['bricklin', 1, 6.0, 6.0]],
 [11, ['engine', 1, 6.0, 6.0]],
 [8, ['door', 1, 5.0, 5.0]],
 [15, ['if', 1, 4.4, 4.4]],
 [7, ['day', 1, 4.3, 4.3]],
 [10, ['early', 1, 4.1, 4.1]]]

If we look at the original content of the first document, we can see that this does a fairly good job of selecting the most important components of a message post.

In [327]:
documents[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

Let's try it with the second post.

In [353]:
calculate_tfidf(vectors[1], document_tokens)[:10]

[[57, ['edu', 2, 7.9, 15.8]],
 [78, ['poll', 2, 6.0, 12.0]],
 [44, ['add', 2, 4.0, 8.0]],
 [59, ['experiences', 2, 3.9, 7.8]],
 [48, ['brave', 1, 7.3, 7.3]],
 [52, ['clock', 2, 3.6, 7.2]],
 [94, ['washington', 2, 3.0, 6.0]],
 [47, ['base', 1, 6.0, 6.0]],
 [88, ['speed', 2, 2.7, 5.4]],
 [43, ['adapters', 1, 5.0, 5.0]]]

In [325]:
documents[1]

"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

### Final Touches

* Changing the term-frequency calculation

There is one last component to TF-IDF, and that is to have the term frequency not be a count of the number of words in a post, but a percentage.  This can come in handy when say looking to match a post with a particular phrase.  For example, if we want to find posts that are most related to the word `speed`, then simply counting up the number of occurrences of speed would bias posts that are longer.  These posts are more likely to have more words in general - so we should discount each occurrence of a word the longer the post.

We can do this by defining term frequency not as a simple count of the words, but as a percentage of each document.  So we redefine term frequency to be: 

$tf = \frac{num\_terms}{|D|}$ 

where $|D|$ is the number of words in the document.

In [385]:
def calculate_tfidf(vector, document_tokens, top = 20):
    idf = inverse_doc_frequency(document_tokens)
    word_scores = [[idx, [dictionary.get(idx), num/len(vector), inverse, (num/len(vector))*inverse]] for (idx, num), inverse in zip(vector, idf)]
    word_scores.sort(key=lambda x: x[-1][-1], reverse=True)
    return word_scores

In [386]:
calculate_tfidf(vectors[0], document_tokens)[:10]

[[6, ['car', 0.09302325581395349, 4.0, 0.37209302325581395]],
 [22, ['lerxst', 0.023255813953488372, 8.6, 0.19999999999999998]],
 [37, ['tellme', 0.023255813953488372, 8.6, 0.19999999999999998]],
 [0, ['addition', 0.023255813953488372, 7.9, 0.18372093023255814]],
 [12, ['enlighten', 0.023255813953488372, 7.3, 0.1697674418604651]],
 [19, ['it', 0.046511627906976744, 3.6, 0.16744186046511628]],
 [2, ['bricklin', 0.023255813953488372, 6.0, 0.13953488372093023]],
 [11, ['engine', 0.023255813953488372, 6.0, 0.13953488372093023]],
 [29, ['neighborhood', 0.023255813953488372, 5.9, 0.1372093023255814]],
 [8, ['door', 0.023255813953488372, 5.0, 0.11627906976744186]]]

* Potentially remove words that only occur once.

### Summary

In this lesson, we saw how to implement TF-IDF, largely from scratch.  Term Frequency Inverse Document Frequency simply multiplies those two components.  

Term frequency is the number of times a given word occurs in a document, divided by the length of the document.  

And inverse document frequency captures how rare a word is throughout the corpus.  We can think of this in two steps: 

1. calculate the frequency of a term throughout the entire corpus $\frac{num t in C}{C}$ which gives a larger number the more frequent a word is

2. Invert this number $\frac{C}{num\_t\_in\_C}$

### Resources

[Sklearn text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

[ML Mastery Multiclass classification](https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/#:~:text=Each%20binary%20classification%20model%20may,one%2Dversus%2Done%20classifier.)